<a href="https://colab.research.google.com/github/EmiljaB/NLP_Projects/blob/News_Modeling/News_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Modeling
#### Emilja Beneja

Topic modeling involves **extracting features from document terms** and using
mathematical structures and frameworks like matrix factorization and SVD to generate **clusters or groups of terms** that are distinguishable from each other and these clusters of words form topics or concepts

Topic modeling is a method for **unsupervised classification** of documents, similar to clustering on numeric data

These concepts can be used to interpret the main **themes** of a corpus and also make **semantic connections among words that co-occur together** frequently in various documents

Topic modeling can help in the following areas:
- discovering the **hidden themes** in the collection
- **classifying** the documents into the discovered themes
- using the classification to **organize/summarize/search** the documents

Frameworks and algorithms to build topic models:
- Latent semantic indexing
- Latent Dirichlet allocation
- Non-negative matrix factorization

## Latent Dirichlet Allocation (LDA)
The latent Dirichlet allocation (LDA) technique is a **generative probabilistic model** where each **document is assumed to have a combination of topics** similar to a probabilistic latent semantic indexing model

In simple words, the idea behind LDA is that of two folds:
- each **document** can be described by a **distribution of topics**
- each **topic** can be described by a **distribution of words**

### LDA Algorithm

- 1. For each document, **randomly initialize each word to one of the K topics** (k is chosen beforehand)
- 2. For each document D, go through each word w and compute:
    - **P(T |D)** , which is a proportion of words in D assigned to topic T
    - **P(W |T )** , which is a proportion of assignments to topic T over all documents having the word W
- **Reassign word W with topic T** with probability P(T |D)´ P(W |T ) considering all other words and their topic assignments

![LDA](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/LDA.png)

### Steps
- Install the necessary library
- Import the necessary libraries
- Download the dataset
- Load the dataset
- Pre-process the dataset
    - Stop words removal
    - Email removal
    - Non-alphabetic words removal
    - Tokenize
    - Lowercase
    - BiGrams & TriGrams
    - Lemmatization
- Create a dictionary for the document
- Filter low frequency words
- Create an Index to word dictionary
- Train the Topic Model
- Predict on the dataset
- Evaluate the Topic Model
    - Model Perplexity
    - Topic Coherence
- Visualize the topics

### Install the necessary library

In [ ]:
!pip install gensim nltk


In [ ]:
!pip install pyLDAvis gensim spacy

# Install spaCy's English language model
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 70.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Import the libraries



In [ ]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pandas as pd

# Load spaCy's English language model
nlp = spacy.load('en_core_web_sm')


### Upload and load the data

In [ ]:
df = pd.read_json("newsgroups.json")
df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


### Preprocess the data

In [ ]:
# Initialize 'processed' column by copying 'content' column
df['processed'] = df['content']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Stop words removal

In [ ]:
def remove_stop_words(text):
    doc = nlp(text)
    return ' '.join([token.text for token in doc if not token.is_stop])

df['processed'] = df['content'].apply(remove_stop_words)
df['processed'].head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,processed
0,: lerxst@wam.umd.edu ( thing ) \n Subject : ca...
1,: guykuo@carson.u.washington.edu ( Guy Kuo ) \...
2,: twillis@ec.ecn.purdue.edu ( Thomas E Willis ...
3,: jgreen@amber ( Joe Green ) \n Subject : : We...
4,: jcm@head-cfa.harvard.edu ( Jonathan McDowell...


### Email Removal

In [ ]:
import re

def remove_emails(text):
    return re.sub(r'\S+@\S+', '', text)

df['processed'] = df['processed'].apply(remove_emails)
df['processed'].head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,processed
0,: ( thing ) \n Subject : car ! ? \n Nntp - Po...
1,: ( Guy Kuo ) \n Subject : SI Clock Poll - Fi...
2,: ( Thomas E Willis ) \n Subject : PB questio...
3,: ( Joe Green ) \n Subject : : Weitek P9000 ?...
4,: ( Jonathan McDowell ) \n Subject : : Shuttl...


### Non-Alphabetic Words Removal

In [ ]:
import re

def clean_text(text):
    # Check if the input is a string; if not, return an empty string
    if not isinstance(text, str):
        return ''

    # Remove non-alphabetic characters (keeps spaces between words)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove specific unwanted terms that appear frequently
    unwanted_terms = ["subject", "organization", "nntp", "posting"]
    text = ' '.join([word for word in text.split() if word.lower() not in unwanted_terms])

    return text

df['processed'] = df['processed'].apply(clean_text)
df['processed'].head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,processed
0,thing car Host racwamumdedu University Marylan...
1,Guy Kuo SI Clock Poll Final Summary Final SI c...
2,Thomas E Willis PB questions Purdue University...
3,Joe Green Weitek P Harris Computer Systems Div...
4,Jonathan McDowell Shuttle Launch Question Smit...


### Tokenize

In [ ]:
def tokenize_text(text):
    doc = nlp(text)
    tokens = []
    for token in doc:
        # Filter out tokens that are whitespace, punctuation, or unwanted terms
        if not token.is_space and not token.is_punct and token.text.lower() not in ["subject", "re", "fw"]:
            tokens.append(token.text.lower())
    return tokens

df['processed'] = df['processed'].apply(tokenize_text)
df['processed'].head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,processed
0,"[thing, car, host, racwamumdedu, university, m..."
1,"[guy, kuo, si, clock, poll, final, summary, fi..."
2,"[thomas, e, willis, pb, questions, purdue, uni..."
3,"[joe, green, weitek, p, harris, computer, syst..."
4,"[jonathan, mcdowell, shuttle, launch, question..."


### Lowercase

In [ ]:
df['processed'] = df['processed'].apply(lambda x: [token.lower() for token in x])
df['processed'].head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,processed
0,"[thing, car, host, racwamumdedu, university, m..."
1,"[guy, kuo, si, clock, poll, final, summary, fi..."
2,"[thomas, e, willis, pb, questions, purdue, uni..."
3,"[joe, green, weitek, p, harris, computer, syst..."
4,"[jonathan, mcdowell, shuttle, launch, question..."


### BiGrams & TriGrams

In [ ]:
from gensim.models import Phrases

# Create bigram and trigram models
bigram = Phrases(df['processed'], min_count=5, threshold=100)
trigram = Phrases(bigram[df['processed']], threshold=100)

# Apply the bigram and trigram models
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

df['processed'] = df['processed'].apply(lambda x: trigram_mod[bigram_mod[x]])
df['processed'].head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,processed
0,"[thing, car, host_racwamumdedu, university, ma..."
1,"[guy_kuo, si, clock, poll, final, summary, fin..."
2,"[thomas, e, willis, pb, questions, purdue_univ..."
3,"[joe, green, weitek_p, harris, computer, syste..."
4,"[jonathan, mcdowell, shuttle_launch, question,..."


### Lemmatization

In [ ]:
def lemmatize_text(text):
    doc = nlp(' '.join(text))
    return [token.lemma_ for token in doc if token.lemma_ != '-PRON-']  # Exclude pronouns

df['processed'] = df['processed'].apply(lemmatize_text)
df['processed'].head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,processed
0,"[thing, car, host_racwamumdedu, university, ma..."
1,"[guy_kuo, si, clock, poll, final, summary, fin..."
2,"[thomas, e, willis, pb, question, purdue_unive..."
3,"[joe, green, weitek_p, harris, computer, syste..."
4,"[jonathan, mcdowell, shuttle_launch, question,..."


### Create a Dictionary for the Document

In [ ]:
from gensim import corpora

# Create a dictionary for the 'processed' text data
dictionary = corpora.Dictionary(df['processed'])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Filter Low-Frequency Words

In [ ]:
# Filter words that are too rare or too common
dictionary.filter_extremes(no_below=5, no_above=0.5)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Create an Index-to-Word Dictionary

In [ ]:
# Create a mapping from index to word
index_to_word = {id: word for word, id in dictionary.token2id.items()}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Train the Topic Model

In [ ]:
from gensim.models import LdaModel

# Create a bag-of-words representation for each document
corpus = [dictionary.doc2bow(text) for text in df['processed']]

# Train the LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=42, passes=10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Predict on the Dataset

In [ ]:
# Predict topics for each document
df['topics'] = df['processed'].apply(lambda x: lda_model[dictionary.doc2bow(x)])
df[['content', 'topics']].head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content,topics
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,"[(0, 0.52360725), (5, 0.06840608), (7, 0.39204..."
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,"[(2, 0.7212475), (3, 0.047728073), (6, 0.16559..."
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,"[(0, 0.5383933), (2, 0.29267898), (3, 0.046087..."
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,"[(0, 0.3160432), (2, 0.5339909), (5, 0.13594544)]"
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,"[(0, 0.7124134), (2, 0.061207075), (3, 0.14005..."


### Evaluate the Topic Model

#### a. Model Perplexity

In [ ]:
# Model Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -8.231719920928807


#### b.Topic Coherence

Lower perplexity values imply better generalization in topic models

In [ ]:
from gensim.models import CoherenceModel

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df['processed'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.565726383475668


Overall, a coherence score of 0.57 suggests the model has a fair balance of meaningful topics.

### Visualize the Topics with pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
vis


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.128751 -0.051753       1        1  15.949292
1     -0.123900 -0.134454       2        1  12.463132
3      0.107488  0.173067       3        1  11.127635
0     -0.034794  0.075241       4        1  10.694652
6      0.021129  0.007236       5        1   9.935457
4     -0.154887 -0.073562       6        1   9.686546
2      0.083420  0.184147       7        1   9.496472
5     -0.077343 -0.052194       8        1   9.173186
7     -0.075860  0.046300       9        1   6.894843
8      0.383498 -0.174028      10        1   4.578785, topic_info=        Term         Freq        Total Category  logprob  loglift
906        x  7633.000000  7633.000000  Default  30.0000  30.0000
62         m  2648.000000  2648.000000  Default  29.0000  29.0000
588      god  3266.000000  3266.000000  Default  28.0000  28.0000
350     file  2554.000000  2554.000000  Default  27.0000  27.0000
750      key  1636.000000  1636.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
88       bit   387.899074  1776.626803  Topic10  -4.9947   1.5620
460        a   309.867911   839.823185  Topic10  -5.2193   2.0867
172      use   351.207074  4711.228835  Topic10  -5.0941   0.4874
4959       q   307.684290  1148.796642  Topic10  -5.2264   1.7663
209   system   291.944382  3514.597464  Topic10  -5.2789   0.5956

[741 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
460       1  0.055964     a
460       2  0.051201     a
460       3  0.064299     a
460       4  0.113119     a
460       5  0.098830     a
...     ...       ...   ...
35        5  0.199708  year
35        6  0.185916  year
35        7  0.007471  year
35        8  0.326142  year
35        9  0.008333  year

[2110 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 4, 1, 7, 5, 3, 6, 8, 9])